# Unit testing

As project become larger the focus on reliability becomes more
important. Broadly, there are two categories of errors you will
encounter in your code:

1. Errors where the input to the application (whethere data from a CSV
   file, interactions from a user of a GUI, or data requested from a
   remote source) is bad. These are errors you cannot prevent but they
   can be handled in your code with Python exceptions. In this way the
   application usability is not interrupted.
2. Errors where your code is not working the way it should. These are
   errors you can absolutely prevent by software testing - testing your
   code runs under multiple scenarios and produces the output you
   expect.

There are four phases that must be considered in software testing:

&nbsp; | Phase | Tools in Python
-|-|-
1 | Unit testing - testing discrete sections of code | `unittest`, `pytest`, `nose`
2 | Integration testing - testing that the different sections of code work together correctly | `tox`
3 | System testing - tests of the application as a whole | `robotframework`
4 | Acceptance testing - ensuring the application meets the requirements of the business | `robotframework`

In this chapter we will look at unit testing in Python - the place
software testing should start.

## Setup

In this chapter you will use the `newster` project, found in the `extras`
folder. Start by copying the `newster` directory into your home
folder:

In [ ]:
%%bash
cp -r extras/newster ~/newster

The project has the following structure:

In [ ]:
├── README.md
├── newster.py
├── test.html
├── tests/
└── urls.txt

You will then use the `cookiecutter` to generate a simple pip
installable library with

In [ ]:
%%bash
cookiecutter -f gh:PythonCharmers/cookiecutter-pipproject

The project name and the app name should be `newster`. You can choose
your own preference for the other options.

Once the project is created:

1. Move `newster.py` into the `newster` package folder (it will be
   along-side an automatically created `__init__.py` file)
2. Update the `requirements.txt` file to include the `newspaper3k` and
   `jinja2` libraries, and make sure to pip install them

When complete you should have a folder structure that looks like
(*hint* use the `tree` command from the command line to visualise this
structure):

In [ ]:
├── LICENSE.md
├── MANIFEST.in
├── README.md
├── dev-requirements.txt
├── docs
│   ├── Makefile
│   ├── make.bat
│   └── source
│       ├── conf.py
│       └── index.rst
├── newster
│   ├── __init__.py
│   └── newster.py
├── requirements.txt
├── setup.cfg
├── setup.py
├── tests
│   ├── __init__.py
│   └── test_sample.py
└── update_docs.sh

## pytest

pytest (formerly known as `py.test`) is a simple unit-testing
framework and automated test runner. It makes it easy to write test
cases in Python using simple ``assert`` statements. (It also supports
finding and running tests that are written in the more verbose style
of the ``unittest`` framework in the Python standard library.)

You will notice that you already have a folder called `tests` in your
project which contains a file, `test_sample.py`. You can run these
tests from the commandline with:

In [ ]:
%%bash
pytest

You should see that the test executes successfully. `pytest` looks for
tests automatically when you run the `pytest` command. It traverses
the directory tree looking for Python modules named either `test_*.py`
(the most common) or `*_test.py`. In those modules `pytest` collects
tests from:

- `test` prefixed test functions or methods outside of class
- `test` prefixed test functions or methods inside `Test` prefixed
  test classes (without an `__init__` method)

If you examine `test_sample.py` you will see a single function:

In [ ]:
def test_pass():
    """Sample Test passing with nose and pytest"""
    assert True, "dummy sample test"

With the name `test_pass` it is collected as a test case and run
automatically when you call `pytest` from the command-line.

### Exercise: Writing a new test function

Ideally rather than having dummy tests that don't do anything you
should write tests that test the code you've written.

1. Create a new file in the `tests` package called `test_newster.py`
2. In the file you will need to import the original `newster` script
   with `from newster import newster`
3. Write a test function called `test_process_url` that passes a URL
   (choose one from the `urls.txt` file) to the `newster.process_url`
   function.
4. Test that the function returns a dictionary containing the keys:
   `url`, `title`, `date`, `img_url` and `authors` keys
   - *Hint* in `pytest` the most common way to write a test that
     passes or fails is to use an `assert` statement, e.g. `assert
     isinstance(data, dict)`

Run the tests using `pytest` from the command line, fixing any errors
you encounter.

## Parameterizing tests

Your test should now work with a single input. Often its much more
useful to test against multiple inputs to ensure the output is
consistent. The simple way to do this would be to use a list of URLs
as input and loop over each one to test that it returned what you
expected. With a bit of thinking you'll see why this is an issue:

- No matter the number of URLs you will only see one unit test in your
  results
- More importantly, since `pytest` uses uncaught exceptions to count
  whether a test has passed or failed, the first URL that fails will
  cause the test to stop, potentially missing errors from later URLs
  (and creating more work in the long run)

Instead you can "parametrize" the tests so that the same test is run
by `pytest` as individual independent tests and where a failure with
one URL will not cause the test to fail for all URLs.

To do this `pytest` has the `@pytest.mark.parametrize` decorator. This
decorator requires two arguments (though there are other options):

1. The function arguments to be parametrized. This can be a string of
   comma separated argument names, or a list of string argument
   names. e.g. `'url,response'` or (for the same effect) `['url',
   'response']`
2. A `list` or `tuple` of parameters, themselves stored as lists or
   tuples ((or `pytest.param` for more complex cases).
   - *Note* the decorator cannot take a generator as input.
   - *Note* even for the case of a single argument a collection of
     parameters is required as `pytest` uses argument expansion to
     pass these parameters into the test.

### Exercise: Parametrizing the URLs

Here you will update the `test_process_url` function to take
parametrized URLs rather than always testing the same URL.

1. Update the `test_process_url` function to include an `url`
   argument.
2. Add the `@pytest.mark.parametrize` decorator to the function.
   - The input argument should be `'url'`
3. Add two URLs as the parameters:
   - https://www.theguardian.com/environment/2020/sep/11/undraining-the-swamp-how-rewilders-have-reclaimed-golf-courses-and-waterways
   - https://www.theguardian.com/books/shortcuts/2015/mar/17/terry-pratchetts-name-lives-on-in-the-clacks-with-hidden-web-code

Rerun the test cases with `pytest`. How many tests have now been run?

## Coverage


Now we have tests working, which means that we can fiddle with the code without
fear of breaking things. But who tests the tests? That is, how do we know that
we are writing good tests? One way to measure testing is by *coverage*, which is
the lines of your code that are executed at least once by your test functions.

In pytest, you measure coverage by using the `--cov .` option. You can also
create a per-file report of *which* lines are not covered with the option
`--cov-report term-missing`.

### Exercise: Measuring the amount of code tested

Measure coverage the code coverage of the above tests. *Hint* you can
limit the code where covereage is measured by setting the name of the
module: `pytest --cov=newster`

Once the test have been run you will see a percentage of how much of
your code you are actually testing. What is it?

To generate a nice report with the coverage data by running `coverage
html`. This will create a series of html pages in the `htmlcov`
folder. Run this report and open `htmlcov/index.html` to see a visual
representation of the code that has been tested.


## Property-based testing


The above tests, while providing some sanity checks, are pretty weak, because
they test only one point in the possible data space. A new paradigm in testing
is called *property-based* testing, which focuses on measuring *invariants* on a
wide array of randomly-generated inputs.

For example, if we were writing *square* and *square-root* functions, we could
test that the square of a number greater than 1 is always bigger than the number
itself, and that the square root of the square of a positive number is equal to
the original number (within some tolerance).

### Hypothesis

Hypothesis is a Python library for property-based testing. We can use it to
encode the above conditions using *strategies* for generating test cases:

In [2]:
from hypothesis import given, strategies

In [11]:
from math import sqrt

def square(x):
    return x * x

@given(strategies.floats(min_value=1))
def test_square(x):
    assert square(x) >= x

In [12]:
test_square()

So what is going on in this testing? One way to tell is to modify the test to
print the value of `x`:

In [13]:
@given(strategies.floats(min_value=1))
def test_square(x):
    print(x, ',', square(x))
    assert square(x) >= x

In [14]:
# Run this:
# test_square()

As you can see, Hypothesis tests not only various random input values, but also
some strategically-chosen floating point values within the constraints: the
maximum floating point value, values extremely close to 1.0, values whose
squares overflow to `inf`, and so on. Thus it can test a much broader range of
input circumstances than ordinary test writing.

### Exercise: Testing using hypothesis

The `newster` library doesn't lend itself to random input - a randomly
generated URL is almost certainly going to be incorrect! Write a test
that checks that the square root of the square of a positive number is
itself. Does it pass Hypothesis's battery of tests?

Modify the strategy to ensure that the tests pass.

## Test fixtures

It's often useful to predefine data and parameters that might be used
for multiple functions. These predefined data are called fixtures. In
`pytest` you can define fixtures so that the outputs are computed at
different stages of the process with the `scope`:

- `function`: the default scope, the fixture is destroyed at the end
  of the test.
- `class`: the fixture is destroyed during teardown of the last test
  in the class.
- `module`: the fixture is destroyed during teardown of the last test
  in the module.
- `package`: the fixture is destroyed during teardown of the last test
  in the package.
- `session`: the fixture is destroyed at the end of the test session.

Some examples of fixtures might include:

- A randomly generated row from a table that is used as the input to a
  function. The can be at any level of scope as it shouldn't matter
  how often it is regenerated.
- A connection to a remote database that contains test data. This
  would be a session level fixture as you would not want to disconnect
  and reconnect in teh middle of each test.
- A temporary folder where you will write output. This may be a module
  fixture to allow a temporary directory for all tests within the
  current module

A fixture is itself a function decorated with `@pytest.fixture` (for
the default) or `@pytest.fixture(scope=scope)` to specify an
individual scope. They can be stored along side the tests themselves,
or they can be stored in a separate module. This is especially useful
if you store them in `conftest.py` as `pytest` looks for this
automatically and loads fixtures (and other configuration) so you do
not need to import it separately.

### Exercise: Adding a temporary directory fixture

1. Create a new file `conftest.py` in the `tests` directory
2. Add a new function `working_directory`. This function should:
   1. Create a new temporary directory on the system (use the
      `tempfile.mkdtemp` function)
   2. Use the `logging` library to log the location of the temporary
      directory at the `info` level
   3. `yield` the name of the temporary directory
   4. Cleanup the directory on completion of the test using
      `shutil.rmtree`
   5. Be decorated with the `@pytest.fixture(scope='function')`
      decorator so that for every

   Note that by calling the script `conftest.py` pytest will
   autodiscover the fixtures, and therefore there is no requirement to
   import the `working_directory` function in to your
   `test_newster.py` script
3. While not necessary for this test, add a new argument
   `working_directory` to the `test_process_url` function.
   - *Hint* This must match exactly with the name of the fixture
     function.
4. Run the tests with `pytest --log-cli-level=INFO`

## Mocking the the results of a function

Of course the fixtures can go much further. For example within a
fixture you can temporarily overwrite (or "monkey-patch") the results
of a function (known as "mocking"). This is extremely useful for
testing functions that rely on the response of another function.

Most importantly mocking should be used to separate tests from your
code from reliance on external libraries / resources. For example if
your test for `newster` is going to call a URL many times via the
`newspaper3k` library it is unecessary for you to test this multiple
times (this would be integration testing).

### Exercise: monkey-patching a function result

Out of the box `pytest` includes a `monkeypatch` fixture designed to
overwrite the value of a function temporarily and then revert once the
function has finished.

1. Write a new `test_generate_report` function in
   `test_newster.py`. It should:
   1. Use the fixtures `working_directory` and `monkeypatch`
   2. Define a new local function `mockreturn`:
      ```python
      def mockreturn(url):
          return {'url': url, 'title': 'Title', 'date': '23-09-2020', 'img_url': None, 'authors': ['Henry', 'Ed']}
      ```
   3. Use the `monkeypatch` fixture to overwrite the `process_url` function:
      ```python
      monkeypatch.setattr(newster, 'process_url', mockreturn)
      ```
   4. Use the `monkeypatch` fixture to change directories to the working directory:
      ```python
      monkeypatch.chdir(working_directory)
      ```
   5. Run the `newster.generate_report` function with:
      - `urls=['http']'`
      - `output=open('report.html', 'wt')`
   6. Assert that `report.html` exists and the file size is greater
      than zero
      - *Hint* look at `os.path.exists` and `os.stat`
2. Once again run the tests with `pytest --log-cli-level=INFO`
3. Check that the temporary directory shown in the logs doesn't exist
   any longer
4. Run `pytest --cov=newster`. What percentage of the module is
   included with the unit test?
5. Parametrize the test so that it tests both the HTML and CSV report
   formats, and run pytest with coverage again. Have you tested 100%
   of the code in this module?


### Mocking with `pytest-mock`

The previous test overwrote one of the functions we had
defined. Python (as of version 3.3) has a built in `unittest.mock`
module which is designed to let you overwrite specific objects. The
`pytest-mock` extension allows you to easily integrate this with your
`pytest` tests by providing a `mocker` fixture you can include in your
tests.

In your test function you can include a `mocker.patch` call to patch a
Python object. For example:

In [ ]:
mocker.patch('newspaper.Article', autospec=True)

This patches `newspaper.Article` by replacing it with a
`unittest.mock.MagicMock` object. *Note* the container object is
required, so if you are patching an object in the same file you're
working from you will either need to reference `__main__.functionname`
(if you're just running the mock tests), or the full module location
of the object `path.to.object`. This is not advised.

The `autospec` keyword argument is extermely powerful in mocking
objects to be used inside functions. From the documentation:

> If you set `autospec=True` then the mock will be created with a spec
> from the object being replaced. All attributes of the mock will also
> have the spec of the corresponding attribute of the object being
> replaced. Methods and functions being mocked will have their
> arguments checked and will raise a `TypeError` if they are called
> with the wrong signature. For mocks replacing a class, their return
> value (the ‘instance’) will have the same spec as the class.

A good example of why this is useful would be to compare these two
calls to the Article object:

In [ ]:
article = Article(url)
article.download()

versus

In [ ]:
article = Article(url)
article.downloa()

On first inspection these look extermely similar. However the second
version is missing the `d` at the end of `download`. If you don't set
a specification when you are mocking an object using `unittest.mock`
then any function call or attribute accessor on that function will be
valid, and the second version would pass your unit test. By having
`unittest.mock` build the specification (with `autospec=True`) then
only the first version of the above would pass.

There is one more trick when dealing with `autospec`: if the
attributes are defined as part of the `__init__` function they will
not be extracted by the autospec. Instead subclass the object to
define required attributes at the top level and use it in place of the
specification:

In [ ]:
from datetime import date

class ArticleForTest(newspaper.Article):
    publish_date = date.today()
    authors = []
    url = ''
    title = ''
    top_image = ''

mocker.patch('newspaper.Article', autospec=ArticleForTest)

### Exercise: mocking with `pytest-mock`

1. Create a new test in `test_newster.py` called
   `test_process_url_with_mock` that takes the `mocker` fixture as an
   argument.
2. Inside the function define the class `ArticleForTest` as above.
3. When mocking instead of using `mocker.patch` directly, use:

   ``` python
   mocker.patch.object(newster, 'Article', autospec=ArticleForTest)
   ```
   This patches the article object as imported in the `newster.py` script
4. Run the process_url function with two URLs:
   - `'bad_url'`
   - `'https://www.theguardian.com/environment/2020/sep/11/undraining-the-swamp-how-rewilders-have-reclaimed-golf-courses-and-waterways'`

Does the test pass?